In [2]:
import pandas as pd
# reading csv files
colTitles = ['age','workclass','fnlwgt','education','education-num'\
             ,'marital-status','occupation','relationship','race','sex'\
             ,'capital-gain','capital-loss','hours-per-week','native-country','income']
data =  pd.read_csv('adult.data', sep=",",names=colTitles)
data = data.drop(columns=['fnlwgt','education-num','relationship','capital-gain','capital-loss'])
print(data)

       age          workclass  ...  native-country  income
0       39          State-gov  ...   United-States   <=50K
1       50   Self-emp-not-inc  ...   United-States   <=50K
2       38            Private  ...   United-States   <=50K
3       53            Private  ...   United-States   <=50K
4       28            Private  ...            Cuba   <=50K
...    ...                ...  ...             ...     ...
32556   27            Private  ...   United-States   <=50K
32557   40            Private  ...   United-States    >50K
32558   58            Private  ...   United-States   <=50K
32559   22            Private  ...   United-States   <=50K
32560   52       Self-emp-inc  ...   United-States    >50K

[32561 rows x 10 columns]


In [3]:
rows = data.shape[0]
for row in range(rows):
  # age: 5 years per class
  data.iloc[row,0] = data.iloc[row,0]//5
  # hours-per-week: 10 hrs per class
  data.iloc[row,7] = data.iloc[row,7]//10
  # income: 1: >50k, 0:<= 50k
  if data.iloc[row,9] == ' <=50K':
    data.iloc[row,9] = 0
  else:
    data.iloc[row,9] = 1
print(data)

# remove all spaces and remove rows that contains ?
for i in range(10):
    if i not in [0,7,9]:
        data.iloc[:,i] = data.iloc[:,i].str.strip()
    data = data[data.iloc[:,i] != '?']

       age          workclass  ...  native-country income
0        7          State-gov  ...   United-States      0
1       10   Self-emp-not-inc  ...   United-States      0
2        7            Private  ...   United-States      0
3       10            Private  ...   United-States      0
4        5            Private  ...            Cuba      0
...    ...                ...  ...             ...    ...
32556    5            Private  ...   United-States      0
32557    8            Private  ...   United-States      1
32558   11            Private  ...   United-States      0
32559    4            Private  ...   United-States      0
32560   10       Self-emp-inc  ...   United-States      1

[32561 rows x 10 columns]


In [4]:
! pip install apyori
import numpy as np
import matplotlib.pyplot as plt
from apyori import apriori


  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=b24d884462724dbc732df919ba8887f81f6acf335887ed7604a54826ae71e2b3
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [5]:
rows = data.shape[0]
cols = data.shape[1]
attributes = list(data.columns)
dataList = []
for row in range(rows):
  oneRow = []
  for col in range(cols):
    oneRow.append(attributes[col]+':'+str(data.iloc[row,col]))
  dataList.append(oneRow)
print(dataList[0:10])

[['age:7', 'workclass:State-gov', 'education:Bachelors', 'marital-status:Never-married', 'occupation:Adm-clerical', 'race:White', 'sex:Male', 'hours-per-week:4', 'native-country:United-States', 'income:0'], ['age:10', 'workclass:Self-emp-not-inc', 'education:Bachelors', 'marital-status:Married-civ-spouse', 'occupation:Exec-managerial', 'race:White', 'sex:Male', 'hours-per-week:1', 'native-country:United-States', 'income:0'], ['age:7', 'workclass:Private', 'education:HS-grad', 'marital-status:Divorced', 'occupation:Handlers-cleaners', 'race:White', 'sex:Male', 'hours-per-week:4', 'native-country:United-States', 'income:0'], ['age:10', 'workclass:Private', 'education:11th', 'marital-status:Married-civ-spouse', 'occupation:Handlers-cleaners', 'race:Black', 'sex:Male', 'hours-per-week:4', 'native-country:United-States', 'income:0'], ['age:5', 'workclass:Private', 'education:Bachelors', 'marital-status:Married-civ-spouse', 'occupation:Prof-specialty', 'race:Black', 'sex:Female', 'hours-per-

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)
rules = apriori(transactions = dataList, min_support = 0.01, min_confidence = 0.5, min_lift = 3, min_length = 4, max_length = 4)
print(rules)
results = list(rules)
results

def inspect(results):
    lhs         = [tuple(result[2][0][0]) for result in results]
    rhs         = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
resultsinDataFrame

<generator object apriori at 0x7f04dc7727d0>


,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,"(age:3,)","(marital-status:Never-married,)",0.044128,0.972243,3.015091
1,"(education:Doctorate,)","(occupation:Prof-specialty,)",0.010013,0.805333,6.015469
2,"(education:Masters,)","(occupation:Prof-specialty,)",0.027153,0.503380,3.760020
3,"(education:Prof-school,)","(income:1,)",0.013461,0.749077,3.009280
4,"(education:Prof-school,)","(occupation:Prof-specialty,)",0.014687,0.817343,6.105177
5,"(age:3, education:11th)","(marital-status:Never-married,)",0.010974,0.982196,3.045958
6,"(age:3, education:Some-college)","(marital-status:Never-married,)",0.011074,0.988166,3.064472
7,"(age:3, hours-per-week:1)","(marital-status:Never-married,)",0.010212,0.993548,3.081165
8,"(age:3, hours-per-week:2)","(marital-status:Never-married,)",0.014290,0.981777,3.044659
9,"(age:3,)","(marital-status:Never-married, income:0)",0.044095,0.971512,3.165811
